In [1]:
year = 2023; day = 5

In [2]:
from aocd import submit
from aocd.models import Puzzle
import numpy as np; import pandas as pd

puzzle = Puzzle(year=year, day=day)
data = puzzle.input_data
# data = puzzle.examples[0].input_data

data = data.strip()
data = data.split("\n\n")

In [3]:
def parse_map(line):
    offsets = []
    name, ranges = line.split(" map:\n")
    ranges =  [list(map(int, r.split())) for r in ranges.split("\n")]
    for r in ranges:
        # print(r)
        d_start, s_start, r_len = r
        delta = d_start - s_start
        x = range(s_start, s_start+r_len)
        offsets.append((x, delta))
    return offsets

In [17]:
def apply_map(m, i):
    for r, d in m:
        if i in r:
            i += d
            break
    return i

def get_location(maps, i):
    for m in maps:
        i = apply_map(m, i)
    return i

In [18]:
def part_1(data):
    maps = [parse_map(d) for d in data[1:]]
    return min([get_location(maps, i) for i in list(map(int, data[0].split()[1:]))])


In [19]:
submit(part_1(data), part="a", year=year, day=day)

aocd will not submit that answer again. At 2023-12-05 00:24:53.671808-05:00 you've previously submitted 650599855 and the server responded with:
That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two]


In [20]:
%timeit part_1(data)

191 µs ± 1.55 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [8]:
def overlap_ranges(a, b):
    matched = []
    unmatched = []
    if a == b:
        return [a], []
    a1, a2 = a.start, a.stop
    bb1, b2 = b.start, b.stop

    if b1 > a1:
        unmatched.append(range(a1, min(a2,b1)))
    if a2 > b2:
        unmatched.append(range(max(a1,b2), a2))
    match_left = max(a1, b1)
    match_right = min(a2, b2)
    if match_left < match_right:
        matched.append(range(match_left, match_right))

    return matched, unmatched

In [10]:
# test overlap_ranges
assert overlap_ranges(range(1, 5), range(5, 7)) == ([], [range(1,5)])
assert overlap_ranges(range(7, 10), range(5, 7)) == ([], [range(7,10)])
assert overlap_ranges(range(7, 10), range(5, 8)) == ([range(7,8)], [range(8,10)])
assert overlap_ranges(range(7, 10), range(5, 8)) == ([range(7,8)], [range(8,10)])
assert overlap_ranges(range(7, 10), range(7, 8)) == ([range(7,8)], [range(8,10)])
assert overlap_ranges(range(7, 10), range(8, 9)) == ([range(8,9)], [range(7,8), range(9,10)])
assert overlap_ranges(range(7, 10), range(7, 10)) == ([range(7,10)], [])
assert overlap_ranges(range(7, 10), range(9, 10)) == ([range(9,10)], [range(7,9)])
assert overlap_ranges(range(79, 93), range(98, 100)) == ([], [range(79,93)])


In [12]:
def add_delta(r, d):
    return range(r.start+d, r.stop+d)

def apply_mappings(m, input_ranges):
    output_matched = []
    unmatched = input_ranges
    print()
    for map_range, delta in m:
        new_unmatched = []
        for r in unmatched:
            matches, unmatches = overlap_ranges(r, map_range)
            matches = [add_delta(r, delta) for r in matches]
            output_matched.extend(matches)
            new_unmatched.extend(unmatches)
        unmatched = new_unmatched
    return output_matched + unmatched


In [21]:
def part_2(data):
    maps = [parse_map(d) for d in data[1:]]


    pairs = list(map(int, data[0].split()[1:]))
    ranges = []
    for i in range(0, len(pairs), 2):
        pair = range(pairs[i], pairs[i]+pairs[i+1])
        ranges.append(pair)

    for m in maps:
        ranges = apply_mappings(m, ranges)

    return min(r.start for r in ranges)

In [22]:
submit(part_2(data), part="b", year=year, day=day)








aocd will not submit that answer again. At 2023-12-05 02:33:42.021176-05:00 you've previously submitted 1240035 and the server responded with:
That's the right answer!  You are one gold star closer to restoring snow operations.You have completed Day 5! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


In [23]:
%timeit part_2(data)